<a href="https://colab.research.google.com/github/momo54/minfo2022/blob/main/LicenceInfo_UE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Licence Info 2022 Analysis and document generation

The overall workflow is:
* get pdf of Licence on the web site
 * PDF (L1/L2/L3 + parcours)
* transform it into excel with ILovePDF
 * https://www.ilovepdf.com/fr/pdf_en_excel

* These two first steps produce a set of excel files. This is the input of notebook.
* Extract All UES from excel file with pandas
 * In this notebook
* Produce ues-uniq.csv for alignement with the ACM classif
 * At this step users can edit the ues-uniq to perform alignement from UE to ACM types
* Extract ECTS and hours from excel file
* produce the radar view thanks to tehe ACM classif


Notes:
Bon... L'extraction excel de pdf ne peut être miraculeuse. Quand un tableau est à cheval sur plusieurs pages, le tableau est mal reconstitué... Du coup, extraire les UEs des Excel est très laborieux...

# install spark for Spark SQL

In [2]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 63.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3ea9e1ac5d6452803e2df11bf0fdae1bc42630e17c6caa834cc40e1bf6395655
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [22]:
import pandas as pd

parcours=[
    ('INFO-info','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx'),
    ('INFO-info','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('INFO-info','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('INFO-palp','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx'),
    ('INFO-palp','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_PALP_2022-2023.xlsx'),
    ('INFO-palp','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('Math-INFO','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Maths_Informatique_2021-2022.xlsx'),
    ('Math-INFO','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Maths_Info_mineure_Maths_Info_2022-2023.xlsx'),
    ('Math-INFO','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__Maths_Info_mineure_CMI_OPTIM_2022-2023.xlsx'),
    ('CMI-INFO','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__CMI_Maths_Informatique_2022-2023.xlsx'),
    ('CMI-INFO','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Maths_Info_mineure_CMI_OPTIM_2022-2023.xlsx'),
    ('CMI-INFO','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__Maths_Info_mineure_CMI_OPTIM_2022-2023.xlsx'),
    ('MIAGE-ALT','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx'),
    ('MIAGE-ALT','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('MIAGE-ALT','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__MIAGE_Alternance_2022-2023.xlsx'),
    ('MIAGE-INFO','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx'),
    ('MIAGE-INFO','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('MIAGE-INFO','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__MIAGE_mineure_MIAGE_Info_2022-2023.xlsx'),
    ('MIAGE-GEST','L1','/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx'),
    ('MIAGE-GEST','L2','/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_Informatique_2022-2023.xlsx'),
    ('MIAGE-GEST','L3','/content/drive/MyDrive/Licence2022/converted/L3_Info__MIAGE_mineure_MIAGE_Gestion_2022-2023.xlsx')    
]

# Extracting Table UES_T from excel Licence Info files

UE with description, parcours, année

In [23]:
import pandas as pd

##
## Autre Approche
## Même si 1 sheet != 1 UE
## l'odre est préservé
## regexp extraction en suivant l'ordre...

my_csv=r'/content/drive/MyDrive/Licence2022/converted/L3_Info__Informatique_mineure_Informatique_2022-2023.xlsx'

df=pd.read_excel(my_csv, index_col=None, header=None, sheet_name="Table 5")
df.rename(columns={0: 'X', 1: 'Y'}, inplace=True)
df=df[df.X.str.contains('X[1-9].*|Objectifs|Contenu',regex=True)]
df


,X,Y
1,X31I010,Etude des algorithmes
14,Objectifs (résultats d'apprentissage),"A l'issue du module, l'étudiant sera initié à\..."
15,Contenu,Analyse de problèmes de décision : présenta...
19,X31A060,Anglais pour la communication scientifique (info)
32,Objectifs (résultats d'apprentissage),"À l’issue de cet enseignement, l’étudiant-e se..."
33,Contenu,1. Développement du vocabulaire scientifique g...


In [24]:
#
# DS M2 et Vico M2 n'ont pas les code UE pour les UE...
# added it by hand in excel files.... -> withcode.xlsx
#


def extract_ue(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]

  for sheet_name, df in pd.read_excel(path, index_col=None, header=None, sheet_name=None).items():
      sheetid=int(sheet_name.split(" ")[1])
      if sheetid>4:
        print(sheet_name)
        dfs.append(df)
  allUE=pd.concat(dfs).reset_index(drop=True)
  allUE.rename(columns={0: 'X', 1: 'Y'}, inplace=True)
  UES=allUE[allUE.X.str.contains('^X[1-4T].*|^ZZ|^YG.*|Object|Contenu',regex=True,na=False)]

#  mask=allUE.apply(lambda x: x.str.contains('^X[1-4T].*|^ZZ|^YG.*|Object|Contenu',regex=True)).any(axis=1)
#  UES=allUE[mask]

  # We should have at least 2 columns
  # just rename in X and Y
  #if len(UES.columns.values)>=2:
  #  UES.rename(columns={0: 'X', 1: 'Y'}, inplace=True)
  #else:
  #  print(f"Found an entry with less than 2 columns:{UES} in {path}")
  #  #UES.set_axis(['X','Y','Z'],axis=1,inplace=True)
    
  #debug
  #print(UES.columns.values)
  #print(UES)
  #for index, row in UES.iterrows():
  #  print(f"extracting:{row['X']}")

  state=0
  key=""
  title=""
  objective=""
  content=""
  lines=[]

  import re
  reg=re.compile("^X[1-4T].*|^ZZ|^YG.*")

  for index, row in UES.iterrows():
    #print(f"extracting:{row['X']}")
    #print(row['X'],row['Y']
    if state==0:
      if reg.match(str(row['X'])) is not None:
        key=row['X']
        title=row['Y']
        print(f"analyzing:{key}:{title}")
        state=1
      else:
        print(f"error: should read an UE key:{row['X']}")
        state=0
    elif state==1:
      if str(row['X']).startswith("Object"):
        objective=row['Y']
        state=2
      else:
        print(f"error: should read an Objective:{row['X']}")
        state=0
    elif state==2:
      if str(row['X']).startswith("Contenu"):
        content=row['Y']
        lines.append({'key' : key, 'title':title,'objective':objective,'content':content,'parc':parcours,'an':an,'from':path})
        state=0
      else:
        print(f"error: should read an Content:{row['X']}")
        state=0
        
  out=pd.DataFrame(lines)
  out.to_csv(f"{parcours}_{an}_UEs.csv",sep='|',index=False,header=True)
  return out

dfs=[]
for p,a,path in parcours:
  df=extract_ue(p,a,path)
  dfs.append(df)

all=pd.concat(dfs)
print(all)


extracting ues for INFO-info, annee:L1, path:/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
Table 25
Table 26
analyzing:X11T010:Méthodologie du Travail Universitaire et Outils Numériques
analyzing:X11A010:Anglais Général
analyzing:X11P020:Base de logique numérique
analyzing:X11X020:Compléments Mathématiques et Informatiques
analyzing:X11P030:Physique : électricité
analyzing:X11T020:Outils de calcul pour les sciences
analyzing:X11G020:Sciences de l'Univers
analyzing:X11I030:Informatique avancée Maths-Info
analyzing:X11M010:Mathématiques avancées pour les sciences
analyzing:X11T020:Outils de calcul pour les sciences
analyzing:X11I040:Bases en informatique maths-Info
analyzing:X11M010:Mathématiques avancées pour les sciences
analyzing:X11I030:Informatique avancée Maths-Info
analyzing

# Writing the files of UEs from the Licence Info 

This file can be edited in Google Sheet to align with ACM classif...

In [25]:
## Work on UE data with SPARK now...

mydata=spark.createDataFrame(all.astype(str))
#mydata.show(5)

mydata.createOrReplaceTempView(f"UES")

ues=spark.sql("select key,title,objective, content, concat_ws(',',collect_set(parc)) as parcours, concat_ws(',',collect_set(an)) as an from UES  group by key,title,objective,content")
ues.show(5) 

ues.createOrReplaceTempView(f"UES_T")

!rm -rf ues-uniq.csv

# Pandas write CSV correctly !!
ues.toPandas().to_csv('ues-uniq.csv')
#ues.write.options(header='True',delimiter='|') \
#  .csv("ues-uniq.csv")

!cp ues-uniq.csv /content/drive/MyDrive/Licence2022


+-------+--------------------+--------------------+--------------------+--------------------+---+
|    key|               title|           objective|             content|            parcours| an|
+-------+--------------------+--------------------+--------------------+--------------------+---+
|X11A010|     Anglais Général|A l’issue de cet ...|L'objectif de cet...|MIAGE-INFO,MIAGE-...| L1|
|X11CI10|Initiation à la t...|                 nan|Ce module entend ...|            CMI-INFO| L1|
|X11G020|Sciences de l'Uni...|Apprentissages de...|L'objectif de cet...|MIAGE-INFO,MIAGE-...| L1|
|X11I030|Informatique avan...|                 nan|                 nan|MIAGE-INFO,MIAGE-...| L1|
|X11I040|Bases en informat...|                 nan|                 nan|MIAGE-INFO,MIAGE-...| L1|
+-------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 5 rows



# Extracting ECTS_T from Excel files

For each UE in every training Course, extract:
UE -> ECTS, Hours

In [27]:
# ok now lets extracts ECTS and hours linked to UE from excel files

def extract_ects(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]

  data=pd.read_excel(path, index_col=None, header=1, sheet_name="Table 2")
  mask=data.apply(lambda x: x.str.contains('^X[1-4T].*|^ZZ|^YG.*',regex=True)).any(axis=1)
  ects=data[mask]
  #print(ects)
  return ects

dfs=[]
for p,a,path in parcours:
  df=extract_ects(p,a,path)
  dfs.append(df)

ects=pd.concat(dfs)
print(ects)



extracting ues for INFO-info, annee:L1, path:/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx
extracting ues for INFO-info, annee:L2, path:/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_Informatique_2022-2023.xlsx
extracting ues for INFO-info, annee:L3, path:/content/drive/MyDrive/Licence2022/converted/L3_Info__Informatique_mineure_Informatique_2022-2023.xlsx
extracting ues for INFO-palp, annee:L1, path:/content/drive/MyDrive/Licence2022/converted/L1_MIP__Informatique_2022-2023.xlsx
extracting ues for INFO-palp, annee:L2, path:/content/drive/MyDrive/Licence2022/converted/L2_Informatique__Informatique_mineure_PALP_2022-2023.xlsx
extracting ues for INFO-palp, annee:L3, path:/content/drive/MyDrive/Licence2022/converted/L3_Info__Informatique_mineure_Informatique_2022-2023.xlsx
extracting ues for Math-INFO, annee:L1, path:/content/drive/MyDrive/Licence2022/converted/L1_MIP__Maths_Informatique_2021-2022.xlsx
extracting ues f

In [28]:
## Work with ECTS with SPARK

#from pyspark.sql.types import *
#ects_schema = StructType([
#    StructField("0", StringType(), True)\
#    ,StructField("1", StringType(), True)
#    ,StructField("2", LongType(), True)
#    ,StructField("3", StringType(), True)
#    ,StructField("4", StringType(), True)
#    ,StructField("5", StringType(), True)
#    ,StructField("6", StringType(), True)
#    ,StructField("8", LongType(), True)
#    ,StructField("9", StringType(), True)
#    ,StructField("10", StringType(), True)
#    ])

s_ects=spark.createDataFrame(ects.astype(str))
#s_ects=spark.createDataFrame(ects,schema=ects_schema)

print(f"Reading {s_ects.count()} in ECTS tablles")

s_ects=s_ects.withColumnRenamed("0","title") \
  .withColumnRenamed("Code","key") \
  .withColumnRenamed("ECTS","ects") \
  .withColumnRenamed("Total","hours") \



s_ects.createOrReplaceTempView(f"ECTS")

ects_table=spark.sql("select key,  cast(max(ects) as float) as ects, cast(max(hours) as float) as hours from ECTS  group by key,ects,hours")
ects_table.createOrReplaceTempView(f"ECTS_T")
ects_table.show(5)

print(f"produced {ects_table.count()} in ECTS_T")


ects_table.toPandas().to_csv('ects.csv')


Reading 559 in ECTS tablles
+-------+----+-----+
|    key|ects|hours|
+-------+----+-----+
|X11A010| 2.0| 17.6|
|X11CI10| 3.0| 26.4|
|X11G020| 3.0| 19.8|
|X11I030| 8.0| 66.0|
|X11I040|11.0| 85.8|
+-------+----+-----+
only showing top 5 rows

produced 132 in ECTS_T


# Extracting ACM_T types alignement from ues_uniq.xlsx

https://docs.google.com/spreadsheets/d/1uTf7f-o-yo17bPnRoW2_0mBzJlG6ILSeQKdzUWs4990/edit?usp=sharing

contains all UEs with alignements to ACM Classif (CS2023)
Extract it and put in table UES_T

In [14]:
# I suppose now that UE has be aligned with ACM classif...
# Read the excel file and group yby ACM items

path='/content/drive/MyDrive/Licence2022/ues-uniq.xlsx'
data=pd.read_excel(path, index_col=None, header=2)

#swtich ain spark now
s_acm=spark.createDataFrame(data.astype(str))
s_acm.show(5)

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum

#s_acm=s_acm.select("key",concat_ws("|",*[when(col(c)=="x",c).alias(c) for c in s_acm.columns]).alias("types"))
# hmmm, bricolage...
s_acm=s_acm.select("key", "title", array_remove(array([when(col(c)=="x",c).otherwise('#').alias(c) for c in s_acm.columns]),'#').alias("types"))

s_acm.show(5,truncate=200)

s_acm.createOrReplaceTempView("ACM_ARRAY")
s_acm=spark.sql("select key, title ,types, size(types) as nbtypes from ACM_ARRAY")

s_acm.show(5,truncate=200)

s_acm=s_acm.select("key",explode("types").alias("types"),"nbtypes")
s_acm.show(100,truncate=200)

s_acm.createOrReplaceTempView("ACM_T")


+----------+-------+--------------------+--------------------+--------------------+--------+---+--------------------------+------------------------------+----------+-----------------------------+---------------------------------+-------------------------------------------------+-----------+----------------------+----------------------+------------------+--------+---------------------------------+-------------+-------------------+-----------------+------------------------+-------------------------------+-------------------------------+------------------------------+-------------------------+----------------------------+------------------------+-------------------------+---------------------------------------+-----------------+-------+------------+
|Unnamed: 0|    key|               title|           objective|             content|parcours| an|T_MSF (Proof, graph proba)|T_AL: Algorithm and Complexity|T_Modeling|T_PL: prog and formal methods|CS_Computer architecture and orga|CS_Concurrent,

# Prepare data for Plotting

In [15]:
# need to compute all types and all prefix to complete results when a training course have not all types or prefix

tmptypes=spark.sql("select types, 0 as ects, 0 as hours from ACM_T group by types")
tmptypes.createOrReplaceTempView("TMPTYPES")
print(f"number of types:{tmptypes.count()}")

tmpprefix=spark.sql("select substring(ACM_T.types,1,2) as prefix, 0 as ects, 0 as hours from ACM_T group by substring(ACM_T.types,1,2)")
tmpprefix.createOrReplaceTempView("TMPPREFIX")
print(f"number of prefix:{tmpprefix.count()}")



number of types:16
number of prefix:4


In [16]:
# Bon plotting now ??

# carefull, we need to have all categories for each parcours !!
# create a temp table

# 1 M1 -> 450h et 60ects
# 1 M2 -> 261h et 60ects
# 1 M1 + M2 ->  711h et 120 ects
# j'ai compté qu'un étudiant faisait toutes les options -> il faut normaliser aux attendus pour que les parcours soient comparables

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum


results_pa={}
for pa,an,path in parcours:
  print(f"parcours:{pa}, année:{an}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by ACM_T.types
  order by ACM_T.types
  """
  
  results_pa[pa]={}
  results_pa[pa+an]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_pa[pa+an].show(26)
  print(results_pa[pa+an].count())
#  results_pa[pa][an].select(sum("ects"),sum("hours")).show()

  nbects=results_pa[pa+an].select(sum('ects')).collect()[0][0]
  nbhours=results_pa[pa+an].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_pa[pa+an]=results_pa[pa+an].withColumn("ects",col("ects")*60/nbects)
  if an=='L1':
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*540/nbhours)
  elif an=='L2':
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*590/nbhours)
  elif an=='L3':
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*510/nbhours)


  results_pa[pa+an].select(sum("ects"),sum("hours")).show()



parc=set([p[0] for p in parcours])

results_p={}
for pa in parc:
  print(f"parcours:{pa}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
  group by ACM_T.types
  order by ACM_T.types
  """
 
  results_p[pa]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_p[pa].show(26)
  print(results_p[pa].count())

  nbects=results_p[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_p[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_p[pa]=results_p[pa].withColumn("ects",col("ects")*180/nbects)
  results_p[pa]=results_p[pa].withColumn("hours",col("hours")*1640/nbhours)

  results_p[pa].select(sum("ects"),sum("hours")).show()




parcours:INFO, année:L1
+--------------------+----+------------------+
|               types|ects|             hours|
+--------------------+----+------------------+
|ACS_Commputer gra...| 0.0|               0.0|
|ACS_Computational...| 8.0| 68.19999885559082|
|ACS_Image and sou...| 0.0|               0.0|
|ACS_Software engi...| 4.0|39.599998474121094|
|ACS_Specialized P...| 0.0|               0.0|
|CS_Computer archi...| 0.0|               0.0|
|CS_Concurrent, pa...| 0.0|               0.0|
|CS_Database and m...| 4.0|39.599998474121094|
|         CS_Networks| 0.0|               0.0|
|   SOC_Language Comm| 6.5| 46.20000076293945|
|             SOC_Pro| 1.0|              11.0|
|SOC_Society, ethi...| 4.0|              33.0|
|T_AL: Algorithm a...| 3.5|30.799999237060547|
|T_MSF (Proof, gra...|58.0| 527.9999961853027|
|          T_Modeling| 0.0|               0.0|
|T_PL: prog and fo...|10.0| 85.79999923706055|
+--------------------+----+------------------+

16
ects:99.0
hours:882.199991226196

In [18]:
results_prefix={}
for pa in parc:
  print(f"prefix parcours:{pa}")

  query=f"""
    select substring(ACM_T.types,1,2) as prefix, 
        sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
        sum(ECTS_T.hours/ACM_T.nbtypes) as hours
    from ACM_T,ECTS_T, UES_T 
    where 
      ACM_T.key=ECTS_T.key 
      and ACM_T.key=UES_T.key 
      and UES_T.parcours like '%{pa}%'
    group by substring(ACM_T.types,1,2) 
    """
  results_prefix[pa]=spark.sql(query).union(tmpprefix).dropDuplicates(["prefix"]).orderBy(["prefix"])
  results_prefix[pa].show(26)
  print(results_prefix[pa].count())

  nbects=results_prefix[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_prefix[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_prefix[pa]=results_prefix[pa].withColumn("ects",col("ects")*180/nbects)
  results_prefix[pa]=results_prefix[pa].withColumn("hours",col("hours")*1640/nbhours)

  results_prefix[pa].select(sum("ects"),sum("hours")).show()



prefix parcours:INFO
+------+-----+------------------+
|prefix| ects|             hours|
+------+-----+------------------+
|    AC| 32.5| 349.0999984741211|
|    CS| 28.0| 266.1799964904785|
|    SO| 40.5| 394.9000024795532|
|    T_|130.0|1204.4999980926514|
+------+-----+------------------+

4
ects:231.0
hours:2214.679995536804
+---------+----------+
|sum(ects)|sum(hours)|
+---------+----------+
|    180.0|    1640.0|
+---------+----------+



# Plotting

In [19]:
# Radar plot Now

import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories=flatten(results_p['INFO'].select("types").collect())

pdata=[]
for p in parc:
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Licence Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

pdata=[]
for p in parc:
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Licence Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")



# plotting prefix

In [21]:
import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories_p=flatten(results_prefix['INFO'].select("prefix").collect())

pdata=[]
for p in parc:
  h=results_prefix[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories_p, name=p,mode="lines+markers+text")
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Licence Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

pdata=[]
for p in parc:
  h=results_prefix[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories_p, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Licence Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")



# plotting per Parcours


In [27]:

# https://plotly.com/python/polar-chart/
for p in parc:

  results_p[p].select(sum("ects"),sum("hours")).show()

  pdata=[]
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()

  print(f"hours:{h}")

  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Licence Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 550]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  pdata=[]
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  print(f"ects:{h}")
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Licence Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 70]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")


+---------+----------+
|sum(ects)|sum(hours)|
+---------+----------+
|    180.0|    1640.0|
+---------+----------+

hours:[56.27901107662698, 115.00171549029858, 11.84821285823726, 69.46014731644912, 5.92410642911863, 50.50300787320313, 16.291292680076232, 96.11121953253948, 34.20430847818342, 66.79430125948788, 88.67646846397375, 136.95793572004888, 240.29656844354145, 521.3213629376054, 17.10585759656345, 113.2244838440464]


ects:[5.454545454545454, 11.2987012987013, 0.7792207792207793, 7.402597402597403, 0.38961038961038963, 5.454545454545454, 1.5584415584415585, 10.909090909090908, 3.896103896103896, 8.96103896103896, 11.2987012987013, 11.2987012987013, 26.493506493506494, 60.0, 1.948051948051948, 12.857142857142858]


In [29]:
# https://plotly.com/python/polar-chart/
for p in parc:

  pdata=[]
  for an in ['L1','L2','L3']:
    h=results_pa[p+an].select("hours").rdd.map(lambda x: x.hours).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Master Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 550]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  pdata=[]
  for an in ['L1','L2','L3']:
    h=results_pa[p+an].select("ects").rdd.map(lambda x: x.ects).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Master Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 70]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")


In [31]:
#Debug, Debug...

#spark.sql("select * from UES_T where UES_T.parcours like '%DS%' and UES_T.an = 'M2'").show()
#spark.sql("select * from ACM_T where key like '%ZZ%'").show() 
#spark.sql("select * from ECTS_T where key like '%ZZ%'").show() 

## CMD + Crt + '+' to comment/uncomment
## Control ATAL
## in fact OK
# pa='ATAL'
# an='M2'
# query=f"""
#   select ACM_T.types, 
#       sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
#       sum(ECTS_T.hours/ACM_T.nbtypes) as hours
#   from ACM_T,ECTS_T, UES_T 
#   where 
#     ACM_T.key=ECTS_T.key 
#     and ACM_T.key=UES_T.key 
#     and UES_T.parcours like '%{pa}%'
#     and UES_T.an = '{an}'
#   group by ACM_T.types
#   order by ACM_T.types
#   """
  
# spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"]).show()

pa='INFO'
an='L3'
query=f"""
  select substring(ACM_T.types,1,2) as prefix, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by substring(ACM_T.types,1,2) 
  """
  
spark.sql(query).show()

+------+----+------------------+
|prefix|ects|             hours|
+------+----+------------------+
|    SO|10.0| 52.80000114440918|
|    T_|22.5|207.90000343322754|
|    CS|15.0|138.57999801635742|
|    AC| 7.5| 69.30000114440918|
+------+----+------------------+



# Generate the competencies file (types-ues.csv)

In [32]:

# types to UES...
# help fill competencies

query=f"""
  select ACM_T.types, 
       "to write !!" as Competencies,
       concat_ws("\n",collect_set(concat_ws(' ',UES_T.title,UES_T.key))) as UEs,
       concat_ws("\n",collect_set(concat_ws("\n",UES_T.title,UES_T.key,UES_T.objective,UES_T.content))) as Objectives
  from ACM_T, UES_T 
  where 
    ACM_T.key=UES_T.key 
  group by ACM_T.types 
  order by ACM_T.types
  """
  
types_to_ues=spark.sql(query)
types_to_ues.toPandas().to_csv('types-ues.csv')

#write also to my drive...
types_to_ues.toPandas().to_csv('/content/drive/MyDrive/MasterInfo2022/types-ues.csv')

types_to_ues.show(5)

+--------------------+------------+--------------------+--------------------+
|               types|Competencies|                 UEs|          Objectives|
+--------------------+------------+--------------------+--------------------+
|ACS_Commputer gra...| to write !!|Arts et Sciences ...|Création numériqu...|
|ACS_Computational...| to write !!|HST : Styles de r...|Climats : passés,...|
|ACS_Image and sou...| to write !!|Techniques d’imag...|Techniques d’imag...|
|ACS_Software engi...| to write !!|Création de pages...|Création de pages...|
|ACS_Specialized P...| to write !!|Création de pages...|Création de pages...|
+--------------------+------------+--------------------+--------------------+
only showing top 5 rows

